In [4]:
import numpy as np
import pandas as pd
#from scipy.stats import truncnorm
from sklearn.preprocessing import PolynomialFeatures



In [5]:

def get_covariance_matrix(p: int, mean_correlation: float=0.3):

    mean = np.zeros(p)
    ''' Get n*n covariance matrix '''
    variance_diagonal = np.ones(p) # in principle, n could be n draws from some meta-distribution
    matrix = np.zeros((p, p))       
    # assign variance to diagonal
    
    while True:
        try:
            for i in range(p): #columns
                matrix[i, i] = variance_diagonal[i]

                for j in range(i+1, p):
                    correlation = np.clip(np.random.normal(mean_correlation, 0.01), -1, 1)
                    #print(f"Correlation: {correlation}")
                    matrix[i, j] = matrix[j, i] = correlation *  np.sqrt(variance_diagonal[i] * variance_diagonal[j])

            pos_def_test = np.linalg.cholesky(matrix)    

        except np.linalg.LinAlgError:
            continue
    
        break


    return matrix, mean

cov = get_covariance_matrix(100, mean_correlation=0.2)


In [6]:
def get_dataframe(cov: np.ndarray, mean: np.ndarray, p: int, n:int = 5000 ) -> np.ndarray:
    
    rng = np.random.default_rng()
    multivariate_samples = rng.multivariate_normal(mean, cov, n)
    df = pd.DataFrame(multivariate_samples, columns=[f"X{i}" for i in range(p)])
    return df

p = 100
cov, mean = get_covariance_matrix(p, mean_correlation=0.2)

df = get_dataframe(cov, mean, p)
df["T"] = np.random.binomial(1, 0.5, len(df))

In [7]:
def gen_mu_x(p: int, df: pd.DataFrame, n: int=5000) -> pd.DataFrame:

    feat_no = int(p/2)

    columns = [f"X{i}" for i in range(feat_no)]
    poly = PolynomialFeatures(interaction_only=True)

    poly_features = poly.fit_transform(df[columns])
    sum_poly_features = poly_features.sum(axis=1)

    df['mu_x'] = sum_poly_features + np.random.normal(0, 1, n)

    return df

gen_mu_x(100, df)
df

c:\Users\joaov\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\joaov\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\joaov\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\joaov\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X92,X93,X94,X95,X96,X97,X98,X99,T,mu_x
0,0.896785,-0.147329,1.849624,2.264488,2.656322,1.349040,1.748846,0.890020,0.382890,0.397965,...,0.375008,1.139504,1.219876,-1.561691,1.118541,2.325467,0.615396,1.146858,0,469.701148
1,-0.997224,-0.340922,0.595889,1.045449,-0.353690,0.788365,1.429754,1.258116,0.414064,0.387576,...,-0.716325,1.318468,0.285078,1.264050,0.110376,0.568027,1.549151,0.642881,1,413.125631
2,0.324607,1.169969,-0.477823,-0.535442,-0.799342,-1.056742,1.017288,1.884238,2.305599,-0.642073,...,0.157227,-0.528425,-1.752789,0.579940,-0.168683,-0.016648,-0.056258,-0.499969,0,-11.232165
3,0.587578,-0.070703,1.446595,-1.375425,0.497588,-0.660880,-0.173774,0.418149,1.038610,-0.356831,...,-0.039341,-1.128276,-0.630754,-0.103615,-0.841686,-0.812886,2.221837,-0.624085,1,22.008514
4,1.183777,0.625599,-0.887260,0.073522,1.369998,0.127298,-0.175880,1.764302,1.973361,-0.577346,...,-0.667267,0.689575,0.610555,-0.084054,0.860640,-0.281721,0.356389,1.008353,1,139.225654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,-0.708038,-1.181963,-0.670345,-0.301262,0.459657,-0.443490,1.849758,0.783997,0.958638,1.435031,...,-1.227546,1.435889,-0.331156,0.520055,-1.068866,1.004148,-2.307777,1.500367,1,-16.104461
4996,-1.102994,0.936661,-1.049443,0.480480,-0.588668,-0.090787,-0.238176,-0.835877,0.547831,0.345719,...,1.776131,-0.496507,1.321264,-1.548812,-0.197953,1.223592,1.067123,0.266135,0,-6.710885
4997,-0.876732,-1.258859,-0.518887,0.471312,0.130302,-1.892160,0.443984,0.248961,-1.407993,0.168267,...,1.421288,0.782697,-0.436262,-0.879678,-1.181884,-0.968175,-0.335456,-0.136940,0,-1.628753
4998,-0.282882,0.154172,0.984893,0.710762,0.886917,1.102068,-1.395179,-1.667285,-0.456614,0.172361,...,-0.243293,-0.261989,1.115123,-0.452041,0.314244,-1.866631,-0.488943,-0.800834,0,-15.521329


In [8]:
def gen_cate(p: int, degree: int, df: pd.DataFrame, n:int = 5000) -> pd.DataFrame: 
    
    feat_no = int(p/2)

    columns = [f"X{i}" for i in range(feat_no)]
    poly = PolynomialFeatures(degree)
    poly_features = poly.fit_transform(df[columns])

    # Sum the polynomial features along axis 1
    sum_poly_features = poly_features.sum(axis=1)

    # Add the new variable to DataFrame
    df['CATE'] = (sum_poly_features + np.random.normal(0, 1, n))*df['T']

    return df

gen_cate(100, 2, df)

c:\Users\joaov\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\joaov\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\joaov\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\joaov\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X93,X94,X95,X96,X97,X98,X99,T,mu_x,CATE
0,0.896785,-0.147329,1.849624,2.264488,2.656322,1.349040,1.748846,0.890020,0.382890,0.397965,...,1.139504,1.219876,-1.561691,1.118541,2.325467,0.615396,1.146858,0,469.701148,0.000000
1,-0.997224,-0.340922,0.595889,1.045449,-0.353690,0.788365,1.429754,1.258116,0.414064,0.387576,...,1.318468,0.285078,1.264050,0.110376,0.568027,1.549151,0.642881,1,413.125631,460.960800
2,0.324607,1.169969,-0.477823,-0.535442,-0.799342,-1.056742,1.017288,1.884238,2.305599,-0.642073,...,-0.528425,-1.752789,0.579940,-0.168683,-0.016648,-0.056258,-0.499969,0,-11.232165,0.000000
3,0.587578,-0.070703,1.446595,-1.375425,0.497588,-0.660880,-0.173774,0.418149,1.038610,-0.356831,...,-1.128276,-0.630754,-0.103615,-0.841686,-0.812886,2.221837,-0.624085,1,22.008514,57.797209
4,1.183777,0.625599,-0.887260,0.073522,1.369998,0.127298,-0.175880,1.764302,1.973361,-0.577346,...,0.689575,0.610555,-0.084054,0.860640,-0.281721,0.356389,1.008353,1,139.225654,192.395204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,-0.708038,-1.181963,-0.670345,-0.301262,0.459657,-0.443490,1.849758,0.783997,0.958638,1.435031,...,1.435889,-0.331156,0.520055,-1.068866,1.004148,-2.307777,1.500367,1,-16.104461,20.011215
4996,-1.102994,0.936661,-1.049443,0.480480,-0.588668,-0.090787,-0.238176,-0.835877,0.547831,0.345719,...,-0.496507,1.321264,-1.548812,-0.197953,1.223592,1.067123,0.266135,0,-6.710885,0.000000
4997,-0.876732,-1.258859,-0.518887,0.471312,0.130302,-1.892160,0.443984,0.248961,-1.407993,0.168267,...,0.782697,-0.436262,-0.879678,-1.181884,-0.968175,-0.335456,-0.136940,0,-1.628753,0.000000
4998,-0.282882,0.154172,0.984893,0.710762,0.886917,1.102068,-1.395179,-1.667285,-0.456614,0.172361,...,-0.261989,1.115123,-0.452041,0.314244,-1.866631,-0.488943,-0.800834,0,-15.521329,0.000000


In [9]:
def gen_outcome(df: pd.DataFrame, n:int = 5000) -> pd.DataFrame:
    df['y'] = df['CATE'] + df['mu_x'] + np.random.normal(0, 1, n)
    return df

gen_outcome(df)

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X94,X95,X96,X97,X98,X99,T,mu_x,CATE,y
0,0.896785,-0.147329,1.849624,2.264488,2.656322,1.349040,1.748846,0.890020,0.382890,0.397965,...,1.219876,-1.561691,1.118541,2.325467,0.615396,1.146858,0,469.701148,0.000000,468.337139
1,-0.997224,-0.340922,0.595889,1.045449,-0.353690,0.788365,1.429754,1.258116,0.414064,0.387576,...,0.285078,1.264050,0.110376,0.568027,1.549151,0.642881,1,413.125631,460.960800,874.030294
2,0.324607,1.169969,-0.477823,-0.535442,-0.799342,-1.056742,1.017288,1.884238,2.305599,-0.642073,...,-1.752789,0.579940,-0.168683,-0.016648,-0.056258,-0.499969,0,-11.232165,0.000000,-11.544209
3,0.587578,-0.070703,1.446595,-1.375425,0.497588,-0.660880,-0.173774,0.418149,1.038610,-0.356831,...,-0.630754,-0.103615,-0.841686,-0.812886,2.221837,-0.624085,1,22.008514,57.797209,79.547010
4,1.183777,0.625599,-0.887260,0.073522,1.369998,0.127298,-0.175880,1.764302,1.973361,-0.577346,...,0.610555,-0.084054,0.860640,-0.281721,0.356389,1.008353,1,139.225654,192.395204,331.204177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,-0.708038,-1.181963,-0.670345,-0.301262,0.459657,-0.443490,1.849758,0.783997,0.958638,1.435031,...,-0.331156,0.520055,-1.068866,1.004148,-2.307777,1.500367,1,-16.104461,20.011215,3.454992
4996,-1.102994,0.936661,-1.049443,0.480480,-0.588668,-0.090787,-0.238176,-0.835877,0.547831,0.345719,...,1.321264,-1.548812,-0.197953,1.223592,1.067123,0.266135,0,-6.710885,0.000000,-6.371396
4997,-0.876732,-1.258859,-0.518887,0.471312,0.130302,-1.892160,0.443984,0.248961,-1.407993,0.168267,...,-0.436262,-0.879678,-1.181884,-0.968175,-0.335456,-0.136940,0,-1.628753,0.000000,-3.767817
4998,-0.282882,0.154172,0.984893,0.710762,0.886917,1.102068,-1.395179,-1.667285,-0.456614,0.172361,...,1.115123,-0.452041,0.314244,-1.866631,-0.488943,-0.800834,0,-15.521329,0.000000,-15.315483
